In [1]:
#!/usr/bin/env python
#import SciServer.LoginPortal as Login
#token=Login.getToken()
#user = Login.getKeystoneUserWithToken(token)
#import SciServer.CasJobs as CasJobs # query with CasJobs
#import SciServer.SciDrive           # read/write to/from SciDrive
import urllib
from bs4 import BeautifulSoup as bs
#import pandas
import numpy as np
import os
from datetime import datetime
import dateutil
import re
import pandas
#import tabulate
#from scipy import stats
#print(token)

trimmer = re.compile(r'^[ \t]+|[ \t]+$')
firstword_capitalized = re.compile(r'([A-Z])\w+')
#capitalized = re.compile('([A-Z][a-zA-Z]*\s*)+')
entryend = re.compile(r'.\s*\[')
oxfordcomma = re.compile(r',\s+(and)')
has_not_number = re.compile(r'\D')

In [2]:
inclusiverange = lambda start, end: range(start, end+1)
datearray = np.empty((24,1),dtype=datetime)
index = 0
for i in inclusiverange(2015,2016):
    for j in inclusiverange(1,12):
        datestring = (str(i)+'-'+str(j)+'-'+'1')
        datearray[index,0] = dateutil.parser.parse(datestring)
        index = index + 1
#newdatearray = datearray[0:22,0:1]   # keep up through October 2016
#print(newdatearray)
newdatearray = datearray[0:22,0:1]
deadpeoplelist = []
linklist = []
thelist = []
thisday = []
#for i in range(0,2):
for i in range(0,len(newdatearray)):
    pagetitle = "Deaths in "+newdatearray[(i,0)].strftime('%B %Y')
    pagetitle2 = str.replace(pagetitle," ","_")
    pageurl = "http://www.wikipedia.org/wiki/"+pagetitle2
    print(pageurl)
    page = urllib.request.urlopen(pageurl).read()
    soup = bs(page, "html.parser")
    headlines = soup.find_all('span',class_='mw-headline')
    listofdays = []
    headlinecount = 0
    for p in headlines:
        if (headlinecount >= 1 and headlinecount <= len(headlines)-2):
            listofdays.append(int(p.get_text()))
        headlinecount = headlinecount + 1
    ultags = soup.find_all('ul')
    ulcount = 0
    for j in ultags:
        nestedlist = 0
        if ((ulcount >= 2) and (ulcount <= len(listofdays)+1)):
            daynum = ulcount - 1
            for k in range(0,len(j)):
                x = j.contents[k]
                if (x != '\n'):
                    if (x.find('ul') != None):
                        nestedlist = 1
                    else:
                        deadpeoplelist.append(x.get_text())
                        thisday.append(dateutil.parser.parse(newdatearray[(i,0)].strftime('%Y-%-m')+'-'+str(daynum)))
                        y = x.find('a',href=True)
                        linklist.append('https://en.wikipedia.org'+y['href'])
        if (nestedlist == 0):
            ulcount = ulcount + 1
deadnamelist = []
deadagelist = []
for q in deadpeoplelist:
    if (trimmer.search(q)):
        q = re.sub(trimmer,'',q)
    deadnamelist.append(q.split(',')[0])
    try:
        deadagelist.append(int(q.split(',')[1]))
#        print(name,' ',age)
    except:
        deadagelist.append(np.nan)
        scrat = q
#        print(scrat)
#for m in range(0,len(deadpeoplelist)):
#    print(m,' ',deadpeoplelist[m],' ',deadnamelist[m],' ',deadagelist[m])

deadpeople = np.empty((len(deadpeoplelist),10),dtype=object)
deadpeople[:,0] = deadpeoplelist
deadpeople[:,1] = linklist
deadpeople[:,2] = thisday
deadpeople[:,3] = deadnamelist
deadpeople[:,4] = deadagelist
print('Done; last row is ',deadpeople[-1])

http://www.wikipedia.org/wiki/Deaths_in_January_2015
http://www.wikipedia.org/wiki/Deaths_in_February_2015
http://www.wikipedia.org/wiki/Deaths_in_March_2015
http://www.wikipedia.org/wiki/Deaths_in_April_2015
http://www.wikipedia.org/wiki/Deaths_in_May_2015
http://www.wikipedia.org/wiki/Deaths_in_June_2015
http://www.wikipedia.org/wiki/Deaths_in_July_2015
http://www.wikipedia.org/wiki/Deaths_in_August_2015
http://www.wikipedia.org/wiki/Deaths_in_September_2015
http://www.wikipedia.org/wiki/Deaths_in_October_2015
http://www.wikipedia.org/wiki/Deaths_in_November_2015
http://www.wikipedia.org/wiki/Deaths_in_December_2015
http://www.wikipedia.org/wiki/Deaths_in_January_2016
http://www.wikipedia.org/wiki/Deaths_in_February_2016
http://www.wikipedia.org/wiki/Deaths_in_March_2016
http://www.wikipedia.org/wiki/Deaths_in_April_2016
http://www.wikipedia.org/wiki/Deaths_in_May_2016
http://www.wikipedia.org/wiki/Deaths_in_June_2016
http://www.wikipedia.org/wiki/Deaths_in_July_2016
http://www.wikip

In [3]:
# columns in deadpeople:
##  0: raw text
##  1: link
##  2: death date
##  3: name (parsed)
##  4: age (parsed)

# Figure out which row numbers have entries we need to fix
rownumberstofix = []
for i in range(0,len(deadpeople[:,0])):
    if (np.isnan(deadpeople[i,4])):    # using NaN for ages that couldn't be parsed as ints
        rownumberstofix.append(i)

print('Number of entries to fix:',len(rownumberstofix))

#special_case_count = 0   # used to count potential special cases in the loop below

# Check whether the special cases will still need to be fixed.

# iterate through deadpeople, but only those rows we identified as needing fixes above
for k in range(0,len(rownumberstofix)):

    # pick a row number and the raw text entry for that row number
    thisrow = rownumberstofix[k]
    textclue = deadpeople[thisrow,0]
    
    # case 1: junior/senior/III
    if ((textclue.split(',')[1] == ' Jr.') or (textclue.split(',')[1] == ' Sr.') or (textclue.split(',')[1] == ' III')):
        deadpeople[thisrow,3] = deadpeople[thisrow,0].split(',')[0]+','+deadpeople[thisrow,0].split(',')[1]
        deadpeople[thisrow,4] = int(deadpeople[thisrow,0].split(',')[2])

    # case 2: age range with an em-dash
    if (textclue.split(',')[1].find('–') != -1):
        splitspot = textclue.split(',')[1].find('–')
        youngage = int(textclue.split(',')[1][1:splitspot])
        oldage = int(textclue.split(',')[1][splitspot+1:9999])
        avgage = (oldage - youngage)/2
        if (avgage == 1.0):
            agey = youngage + 1   # if age has a 2-year range, pick the middle
        elif (avgage == 0.5):
            agey = np.round(youngage+np.random.rand(),0) # if 1-year, one or the other
        else:
            print('wtf? age text is',textclue.split(',')[1])
        deadpeople[thisrow,4] = int(agey)

    # case 3: circa age
    if (textclue.split(',')[1].find('c. ') != -1):
        splitspot = textclue.split(',')[1].find('c. ')
        agey = textclue.split(',')[1][splitspot+3:9999]
#        print(textclue.split(',')[1])
        deadpeople[thisrow,4] = int(agey)

    # case 4: age followed by question mark
    if (textclue.split(',')[1].find('?') != -1):
        splitspot = textclue.split(',')[1].find('?')
        agey = textclue.split(',')[1][1:splitspot]
#        print(textclue.split(',')[1])
#        print(agey)
        deadpeople[thisrow,4] = int(agey)

    # case 5: stupid nobility with commas in their names
    isbaron = (textclue.split(',')[1].find('Baron') != -1)
    ismarquess = (textclue.split(',')[1].find('Marquess') != -1)
    ismarchoiness = (textclue.split(',')[1].find('Marchioness') != -1)
    isearl = (textclue.split(',')[1].find('Earl') != -1)
    isprince = (textclue.split(',')[1].find('Prince') != -1)
    iscount = (textclue.split(',')[1].find('Count') != -1)
    isviscount = (textclue.split(',')[1].find('Viscount') != -1)
    isduke = (textclue.split(',')[1].find('Duke') != -1)
    isduchess = (textclue.split(',')[1].find('Duchess') != -1)
    isseigneur = (textclue.split(',')[1].find('Seigneur') != -1)
    islord = (textclue.split(',')[1].find('Lord') != -1)
    islady = (textclue.split(',')[1].find('Lady') != -1)
    
    isnoble = (isbaron or ismarquess or ismarchoiness or isearl or isprince or iscount or isviscount or isduke or isduchess or isseigneur or islord or islady)
    if (isnoble):
        #print(textclue.split(',')[1])
        deadpeople[thisrow,3] = textclue.split(',')[0]+','+textclue.split(',')[1]
        deadpeople[thisrow,4] = int(textclue.split(',')[2])

    # case 6: some special case idiots

#    special_case_print_flag = 0
#    if (textclue.split(',')[0] in ('Alexis FitzGerald', 'Syed Rizwan Farook','Tashfeen Malik','Leonidas B. Young','Albert Rhoton','Robbie Brennan','Hafsa Mossi','Jimmy Levine')):
#        print(deadpeople[thisrow,0])
#        special_case_count = special_case_count + 1
#        special_case_print_flag = 1
#    if (textclue.split(',')[0] in ('M. K. Wren','Don Brothwell','Abu Mohammad al-Adnani','Shawshank tree','Neelabh Ashk','Irving Sablosky. 92')):
#        print(deadpeople[thisrow,0])
#        special_case_count = special_case_count + 1
#        special_case_print_flag = 1        
#    if (special_case_print_flag == 1):
#        print("Need to fix",special_case_count,"special case idiots in step 6 below.")

    if (textclue.split(',')[0] == 'Alexis FitzGerald'):
        deadpeople[thisrow,3] = 'Alexis FitzGerald, Jnr'
        deadpeople[thisrow,4] = 70
    if (textclue.split(',')[0] == 'Syed Rizwan Farook'):
        deadpeople[thisrow,4] = 28
    if (textclue.split(',')[0] == 'Tashfeen Malik'):
        deadpeople[thisrow,4] = 29
    if (textclue.split(',')[0] == 'Leonidas B. Young'):
        deadpeople[thisrow,3] = 'Leonidas B. Young, II'
        deadpeople[thisrow,4] = 62
    if (textclue.split(',')[0] == 'Albert Rhoton'):
        deadpeople[thisrow,3] = 'Albert Rhoton, Jr'
        deadpeople[thisrow,4] = 83
    if (textclue.split(',')[0] == 'Robbie Brennan'):
        deadpeople[thisrow,4] = np.round(68+np.random.rand(),0) # if 1-year, one or the other
    if (textclue.split(',')[0] == 'Hafsa Mossi'):
        deadpeople[thisrow,4] = np.round(51+np.random.rand(),0) # if 1-year, one or the other
    if (textclue.split(',')[0] == 'Jimmy Levine'):
        deadpeople[thisrow,4] = np.round(61+np.random.rand(),0) # if 1-year, one or the other
    if (textclue.split(',')[0] == 'M. K. Wren'):
        deadpeople[thisrow,4] = np.round(77+np.random.rand(),0) # if 1-year, one or the other
    if (textclue.split(',')[0] == 'Don Brothwell'):
        deadpeople[thisrow,4] = np.round(82+np.random.rand(),0) # if 1-year, one or the other
    if (textclue.split(',')[0] == 'Abu Mohammad al-Adnani'):
        deadpeople[thisrow,4] = np.round(38+np.random.rand(),0) # if 1-year, one or the other
    if (textclue.split(',')[0] == 'Shawshank tree'):
        deadpeople[thisrow,4] = 180
    if (textclue.split(',')[0] == 'Neelabh Ashk'):
        deadpeople[thisrow,4] = 70
    if (textclue.split(',')[0] == 'Irving Sablosky. 92'):
        deadpeople[thisrow,3] = 'Irving Sablosky'
        deadpeople[thisrow,4] = 92

# Done! What still needs to be fixed? At this point, should be just those with unreported ages.
rownumbersstilltofix = []
for i in range(0,len(deadpeople[:,0])):
    if (np.isnan(deadpeople[i,4])):
        rownumbersstilltofix.append(i)
        
print('Number of unreported ages remaining after fixes:',len(rownumbersstilltofix))

# And lastly, print out those entries with unreported ages (now stored as -9999)
#for i in range(0,len(rownumbersstilltofix)):
#    thatrow = rownumbersstilltofix[i]
#    print(deadpeople[thatrow,0],':',deadpeople[thatrow,3])
# columns in deadpeople:
##  0: raw text
##  1: link
##  2: death date
##  3: name (parsed)
##  4: age (parsed)

print('Total entries:',len(deadpeople))

Number of entries to fix: 254
Number of unreported ages remaining after fixes: 55
Total entries: 11806


In [5]:
mask = np.ones(len(deadpeople),dtype=bool)
#print(mask)

animalwords = []
animalwords.append(re.compile(r'cat\b'))
animalwords.append(re.compile(r'frog\b'))
animalwords.append(re.compile(r'dog\b'))
animalwords.append(re.compile(r'horse\b'))
animalwords.append(re.compile(r'tree\b'))
animalwords.append(re.compile(r'parrot\b'))
animalwords.append(re.compile(r'tigress\b'))
animalwords.append(re.compile(r'mascot\b'))
animalwords.append(re.compile(r'canine\b'))
animalwords.append(re.compile(r'rhinoceros\b'))
animalwords.append(re.compile(r'gorilla\b'))
animalwords.append(re.compile(r'cheetah\b'))
animalwords.append(re.compile(r'penguin\b'))
animalwords.append(re.compile(r'panda\b'))


for i in range(0,len(deadpeople)):
    for j in range(0,len(animalwords)):
        if(animalwords[j].search(deadpeople[i,0])):
            mask[i] = 0
        
#Some exceptions to fix!
for k in range(0,len(deadpeople)):
    if (deadpeople[k,3] == 'Chief Zee'):  # human mascot
        mask[k] = 1
    if (deadpeople[k,3] == 'George Attla'):  # dog musher
        mask[k] = 1
    if (deadpeople[k,3] in ('H. Allen Jerkens','Jim Mitchell','Sir Peter O\'Sullevan','John A. Nerud','Bernard Secly','Wallace Dollase','Jack Brizendine','Alan Lee','Sheila Varian','Ogden Mills Phipps','Charles Brantley','Raymond Mould','Bart Cummings','Phonsie O\'Brien','Thomas R. McCarthy')): # horse breeders, trainers, etc.
        mask[k] = 1
    if (deadpeople[k,3] == 'Khalid Mahmood Mithu'): # killed by falling tree
        mask[k] = 1
    if (deadpeople[k,3] in ('André Turcat','Antonella Steni')): # name or description contains "cat"
        mask[k] = 1

#Bart Cummings, 87, Australian racehorse trainer, twelve-time winner of the Melbourne Cup.[437]
#Phonsie O'Brien, 86, Irish jockey and racehorse trainer.[70]
#Thomas R. McCarthy, 82, American racehorse trainer and owner (General Quarters), melanoma.[341]
        
number_masked = 0
for i in range(0,len(mask)):
    if (mask[i] == False):
        number_masked = number_masked + 1
print('Animals and masked out:',number_masked)

for j in range(0,len(deadpeople)):
    if (mask[j] == 1):  
        if ((str(deadpeople[j,1]).find('redlink') != -1) or (str(deadpeople[j,1]).find('#cite_note-') != -1)):
            mask[j] = 0
            
number_masked = 0
for i in range(0,len(mask)):
    if (mask[i] == False):
        number_masked = number_masked + 1
print('Animals and bad links masked out:',number_masked)

#print('\n')
#print('to be masked out:')
#for i in range(0,len(deadpeople)):
#    if (not(mask[i])):
#        print(deadpeople[i,0])

deadpeople = deadpeople[mask]

print('Total remaining:',len(deadpeople))

#print('\n')
#print('staying in:')
#for i in range(0,50):       
#for i in range(0,len(deadpeople)):
#    if (not(mask[i])):
#        print(deadpeople[i,0])



Animals and masked out: 68
Animals and bad links masked out: 125
Total remaining: 11681


In [6]:
# columns in deadpeople:
##  0: raw text
##  1: link
##  2: death date
##  3: name (parsed)
##  4: age (parsed)

# First, trim leading and trailing whitespace from text descriptions
deadpeople[:,0] = [re.sub(trimmer,'',k) for k in deadpeople[:,0]]
deadpeople[:,3] = [re.sub(trimmer,'',k) for k in deadpeople[:,3]]

for i in range(0,len(deadpeople)):
    total_text_length = len(deadpeople[i,0])
    name_length = len(deadpeople[i,3])
    
    removename = deadpeople[i,0][name_length+1:total_text_length]

    if (removename.find(',') <= 6):
        descriptor = removename[removename.find(',')+1:len(removename)]        
    else:
        descriptor = deadpeople[i,0][deadpeople[i,0].find(',')+1:len(deadpeople[i,0])]
    descriptor = re.sub(trimmer,'',descriptor)
    # Step 1: if the first word is capitalized, that's the nationality
    try:
        nationality = descriptor[0:re.match(firstword_capitalized,descriptor).end()]
    except:
        nationality = ''
        
    # Step 2: unless they're born somewhere else
    if (descriptor.find('-') != -1):        
        if (descriptor.find('-born') != -1):
            birthnation = descriptor[0:descriptor.find('-born')+5]
            nextwords = descriptor[descriptor.find('-born')+5:len(descriptor)]
            nextwords = re.sub(trimmer,'',nextwords)
            try:
                nationality = birthnation + ' ' + nextwords[0:re.match(firstword_capitalized,nextwords).end()]
            except:
                nationality = birthnation
        else:
            if (descriptor[len(nationality):len(descriptor)][0] == '-'):
                nationality = descriptor.split(' ')[0]
                
        
    # Step 3: fix two-word nationalities
    for k in ('New','South','North','East','Sri','Puerto','Costa','Hong','Sierra','Cape','Cook','El','Saint','Central','Northern','Solomon','Western'):
        if (nationality.find(k) != -1):
            try:
                nextwords = descriptor[re.search(nationality,descriptor).end():len(descriptor)]
                nextwords = re.sub(trimmer,'',nextwords)
                nationality = nationality + ' '+ nextwords[0:re.match(firstword_capitalized,nextwords).end()]
            except:
                nationality = nationality

    # Step 4: fix Papua New Guinea and Trinidad and Tobago
    if (nationality == 'Papua'):
        nextwords = descriptor[re.search(nationality,descriptor).end():len(descriptor)]
        nextwords = re.sub(trimmer,'',nextwords)
        nationality = nationality + ' '+ nextwords[0:re.match(firstword_capitalized,nextwords).end()]        
        nextwords = descriptor[re.search(nationality,descriptor).end():len(descriptor)]
        nextwords = re.sub(trimmer,'',nextwords)
        nationality = nationality + ' '+ nextwords[0:re.match(firstword_capitalized,nextwords).end()]        
        
    #print(nationality)
    if (nationality.find('Saudi') != -1):
        nextwords = descriptor[re.search(nationality,descriptor).end():len(descriptor)]
        nextwords = re.sub(trimmer,'',nextwords)
        if (nextwords.find('Arabian') != -1):
            nationality = nationality + ' Arabian'

    if (nationality.find('Dominican') != -1):
        nextwords = descriptor[re.search(nationality,descriptor).end():len(descriptor)]
        nextwords = re.sub(trimmer,'',nextwords)
        if (nextwords.find('Republic') != -1):
            nationality = nationality + ' Republic'

    # Special case: ex-Yugoslavia
    if ((descriptor.find('Yugoslav') != -1) & (nationality != 'Yugoslav')):
        nextwords = descriptor[len(nationality)+1:len(descriptor)]
        if (nextwords[0:8] == 'Yugoslav'):
            nationality = nationality + ' ' + nextwords[0:8]

    # Special case: ex-USSR
    if ((descriptor.find('Soviet') != -1)):
        if (nationality == 'Soviet'):
            nextwords = descriptor[7:len(descriptor)]
            if (re.search(firstword_capitalized,nextwords)):
                if (nextwords[0:re.search(firstword_capitalized,nextwords).end()].find('Russian') != -1):
                    nationality = nationality + ' ' + nextwords[0:re.search(firstword_capitalized,nextwords).end()]
                if (nextwords[0:re.search(firstword_capitalized,nextwords).end()].find('Ukrainian') != -1):
                    nationality = nationality + ' ' + nextwords[0:re.search(firstword_capitalized,nextwords).end()]
        else:
            nextwords = descriptor[len(nationality)+1:len(descriptor)]
            if (nextwords[0:7] == 'Soviet '):
                nationality = nationality + ' ' + nextwords[0:6]

    # Special case: Spanish-Catalan and Spanish-Basque
    if (descriptor.find('Spanish') != -1):
        if (descriptor.find('Catalan') != -1):
            nationality = nationality + ' Catalan'
        if (descriptor.find('Basque') != -1):
            nationality = nationality + ' Basque'
    deadpeople[i,5] = nationality

# some special cases
deadpeople[:,5][deadpeople[:,5] == 'Author'] = ''
deadpeople[:,5][deadpeople[:,5] == 'Heavy'] = 'British'
deadpeople[:,5][deadpeople[:,5] == 'Polish-born Holocaust'] = 'Polish-born'

deadpeople[:,5][deadpeople[:,3] == 'Tetaua Taitai'] = 'i-Kiribati'
deadpeople[:,5][deadpeople[:,3] == 'Julio Brady'] = 'U.S. Virgin Islander'
deadpeople[:,5][deadpeople[:,3] == 'Neelabh Ashk'] = 'Indian'
deadpeople[:,5][deadpeople[:,3] == 'Lincoln Myers'] = 'Trinidad and Tobago'
deadpeople[:,5][deadpeople[:,3] == 'Makandal Daaga'] = 'Trinidad and Tobago'
deadpeople[:,5][deadpeople[:,3] == 'Laurence Lerner'] = 'South African-born British'
deadpeople[:,5][deadpeople[:,3] == 'Theodore van Houten'] = 'Dutch-British'
deadpeople[:,5][deadpeople[:,3] == 'Károly Horváth'] = 'Romanian-Hungarian'
deadpeople[:,5][deadpeople[:,3] == 'Sir John Yocklunn'] = 'Australian-Papua New Guinean'
deadpeople[:,5][deadpeople[:,3] == 'Teatao Teannaki'] = 'i-Kiribati'
deadpeople[:,5][deadpeople[:,3] == 'Raymond Gilmour'] = 'Northern Irish'

print('done')

#x = pandas.DataFrame(deadpeople[:,[0,3,5]],columns=['text','name','nationality'])

#x.sort_values('nationality')
#x[x['nationality'] == '']
#y = x.groupby('nationality').size()
#y[(y >= 5) & (y < 10)].sort_values(ascending=False)
#y[y == 1][50:100]

done


In [7]:
# columns in deadpeople:
##  0: raw text
##  1: link
##  2: death date
##  3: name (parsed)
##  4: age (parsed)
##  5: nationality (parsed)
##  6: cause of death

commoncauses = []
# common natural causes
commoncauses.append('heart attack')
commoncauses.append('stroke')
commoncauses.append('diabetes')
commoncauses.append('cancer')
commoncauses.append('cardiac arrest')
commoncauses.append('lymphoma')
commoncauses.append('embolism')
commoncauses.append('pneumonia')
commoncauses.append('leuk')  #leukemia/leukaemia
commoncauses.append('sclerosis')
commoncauses.append('dementia')
commoncauses.append('emphysema')
commoncauses.append('natural causes')
commoncauses.append('seizure')
# violent causes
commoncauses.append('suicide')
commoncauses.append('burn')
commoncauses.append('airstrike')
commoncauses.append('air strike')
commoncauses.append('shot')
commoncauses.append('executed')
commoncauses.append('behead')
commoncauses.append('collision')
commoncauses.append('crash')
commoncauses.append('hanging')
commoncauses.append('stabbed')
commoncauses.append('drown')
#commoncauses.append('murdered')
commoncauses.append('drone')
commoncauses.append('hit by')
commoncauses.append('severed')
commoncauses.append('earthquake')
commoncauses.append('asphyxiation')
commoncauses.append('euthanasia')
commoncauses.append('overdose')
commoncauses.append('intoxication')
commoncauses.append('stampede')
commoncauses.append('beaten')
commoncauses.append('homicide')
commoncauses.append('strangled')
commoncauses.append('explosion')
commoncauses.append('electrocution')
commoncauses.append('building collapse')
commoncauses.append('struck by')
commoncauses.append('gored')
commoncauses.append('bomb')
# less common natural causes
commoncauses.append('dengue')
commoncauses.append('bleeding')
commoncauses.append('malaria')
commoncauses.append('mesothelioma')
commoncauses.append('septic shock')
commoncauses.append('progressive supranuclear palsy')
commoncauses.append('aneur')  #aneurysm/#aneurism
commoncauses.append('morrha') #haemorrhage/hemorrhage
commoncauses.append('myeloma')
commoncauses.append('fibrosis')
commoncauses.append('edema')
commoncauses.append('hypothermia')
commoncauses.append('blood clot')
commoncauses.append('melanoma')
commoncauses.append('enceph') #encephalitis
commoncauses.append('infect') 
commoncauses.append('aortic dissection')
commoncauses.append('ALS')
commoncauses.append('hematoma')
commoncauses.append('pancreatitis')
commoncauses.append('sepsis')
commoncauses.append('sarcoma')
commoncauses.append('asthma')
commoncauses.append('pulmonary distress')
commoncauses.append('respiratory')
commoncauses.append('cerebral')
commoncauses.append('bronchospasm')
commoncauses.append('COPD')
commoncauses.append('jaundice')
# words indicating death but not specifically
commoncauses.append('disease')
commoncauses.append('syndrome')
commoncauses.append('complication')
commoncauses.append('illness')
commoncauses.append('ailment')
commoncauses.append('failure')
commoncauses.append('disorder')
commoncauses.append('killed')
commoncauses.append('injur') # injury/injuries
commoncauses.append('incident')
commoncauses.append('atroph')
commoncauses.append('accident')
# More found on the second run-through
commoncauses.append('adenocarcinoma')
commoncauses.append('execution')
commoncauses.append('anaphylaxis')
commoncauses.append('tumo')
commoncauses.append('fall')
commoncauses.append('meningitis')
commoncauses.append('fire')
commoncauses.append('sarcoidosis')
commoncauses.append('babesi') #babesiosis
commoncauses.append('trauma')
commoncauses.append('bludgeon')
commoncauses.append('missile')
commoncauses.append('deficiency')
commoncauses.append('amyloid')
commoncauses.append('avalanche')
commoncauses.append('glioblastoma')

cause = []
for i in range(0,len(deadpeople)):
    cause.append('')
#for i in range(0,100):
for i in range(0,len(deadpeople)): 
    thelast = deadpeople[i,0].split(',')[len(deadpeople[i,0].split(','))-1]
    thelast = thelast[0:thelast.find('[')]
    thelast = re.sub(trimmer,'',thelast)
    if (thelast[-1] == '.'):
        thelast = thelast[0:-1]
    
    notdeadyet = 0
    j = 0
    while ((notdeadyet == 0) & (j < len(commoncauses))):
        thiscause = commoncauses[j]
        if (thelast.find(thiscause) != -1):
            cause[i] = thelast
            notdeadyet = 1
#            if (j == 15): # highest j currently 100
#                print(i,cause[i],commoncauses[j])
        else:
            j = j + 1
deadpeople[:,6] = cause

# ROWS TO FIX:  
deadpeople[:,6][deadpeople[:,6] == 'Skyfall)'] = ''
deadpeople[:,6][deadpeople[:,6] == 'English newspaper executive and bomber pilot'] = ''
deadpeople[:,6][deadpeople[:,6] == 'American chemist and fire investigator'] = ''
deadpeople[:,6][deadpeople[:,6] == 'discovered the link between phenacetin and kidney cancer'] = ''
deadpeople[:,6][deadpeople[:,6] == 'survivor of National Airlines Flight 101 crash'] = ''
deadpeople[:,6][deadpeople[:,6] == 'American physician known for alternative cancer treatments'] = ''
deadpeople[:,6][deadpeople[:,6] == 'American trauma surgeon'] = ''
deadpeople[:,6][deadpeople[:,6] == 'American bomber crewman'] = ''
deadpeople[:,6][deadpeople[:,6] == 'American World War II bomber pilot'] = ''
deadpeople[:,6][deadpeople[:,6] == 'prematurely announced the fall of the Berlin Wall'] = ''
deadpeople[:,6][deadpeople[:,6] == 'New Zealand paediatrician and sudden infant death syndrome researcher'] = ''
deadpeople[:,6][deadpeople[:,6] == 'Greek shot putter'] = ''
deadpeople[:,6][deadpeople[:,6] == 'Queensland MP for Mackenzie (1956–1972) and Auburn (1972–1980)'] = ''
deadpeople[:,6][deadpeople[:,6] == 'American football player (Auburn Tigers)'] = ''
deadpeople[:,6][deadpeople[:,6] == 'Polish-born Swedish actress (The Slingshot)'] = ''
deadpeople[:,6][deadpeople[:,6] == 'witness in the Lockerbie bombing case'] = ''
print('done')


# OK, there really are few people who died in Oct 2016 who have recorded causes of death.
#x = pandas.DataFrame(deadpeople[:,[0,3,6]],columns=['text','name','cause'])
#x.sort_values('cause')
#z = [str(k) for k in x['text'][x['cause'] == '']]
#z.sort()
#for j in z:
#    print(j)
#y = x.groupby('cause').size()
#y.sort_values(ascending=False)
#y[(y >= 5) & (y < 10)].sort_values(ascending=False)
#y[y == 1]

done


In [8]:
for i in range(0,len(deadpeople)):
    description = ''
    total_text_length = len(deadpeople[i,0])
    name_length = len(deadpeople[i,3])
    
    removename = deadpeople[i,0][name_length+1:total_text_length]

    if (removename.find(',') <= 6):
        descriptor = removename[removename.find(',')+1:len(removename)]        
    else:
        descriptor = deadpeople[i,0][deadpeople[i,0].find(',')+1:len(deadpeople[i,0])]
    descriptor = re.sub(trimmer,'',descriptor)

    nationality_length = len(deadpeople[i,5])
    descriptor = descriptor[nationality_length:len(descriptor)]
    if (re.search(entryend,descriptor)):
        descriptor = descriptor[0:re.search(entryend,descriptor).start()]
    #print(descriptor)
    if (deadpeople[i,6] != ''):
        splitter = descriptor.split(',')
        descriptor = ''
        for k in range(0,len(splitter)-1):
            descriptor += splitter[k]
    re.sub(trimmer,'',descriptor)
    deadpeople[i,7] = descriptor

deadpeople[:,7][deadpeople[:,7] == 'hern Irish undercover agent, infiltrated INLA and PIRA'] = 'undercover agent, infiltrated INLA and PIRA'
deadpeople[:,7][deadpeople[:,7] == '-Canadian Chaldean Catholic hierarch, Archbishop of Ahwaz (1974–2011) and Mar Addai of Toronto (2011–2014)'] = 'Catholic hierarch, Archbishop of Ahwaz (1974–2011) and Mar Addai of Toronto (2011–2014)'

print('Done')

#x = pandas.DataFrame(deadpeople[:,[0,3,7]],columns=['text','name','description'])
#x.sort_values('description')
#x[x['description'] == 'hern Irish undercover agent, infiltrated INLA and PIRA']
#y = x.groupby('description').size()
#y.sort_values(ascending=False)
#y[(y >= 5) & (y < 10)].sort_values(ascending=False)
#y[y == 1][330:380]

Done


In [15]:
deathsurveypandas = pandas.read_csv('deathsurvey.csv',encoding="utf-8")
deathsurvey = deathsurveypandas.values
surveyarray = np.zeros((len(deadpeople),2),dtype=int)

for i in range(0,len(deathsurvey[:,0])):
#    print(i,deathvotes[i,0])
    foundyet = False
    j = 0
    while ((not(foundyet)) and (j < len(deadpeople))):
        if (deadpeople[j,3] == deathsurvey[i,0]):
            surveyarray[j,0] = deathsurvey[i,1]
            surveyarray[j,1] = i
            foundyet = True
        else:
            j = j + 1
#print(surveyarray)
deadpeople[:,8] = surveyarray[:,0]
deadpeople[:,9] = surveyarray[:,1]
print(deadpeople[:,9][deadpeople[:,8] > 0])

[101 7 182 46 52 60 166 196 185 94 114 27 156 95 138 190 148 82 113 75 56
 66 135 23 197 121 145 149 19 47 128 131 174 118 33 172 110 81 107 177 165
 22 139 186 159 83 70 178 176 127 39 21 80 74 103 144 11 183 136 191 180
 153 142 194 98 68 58 29 45 120 9 37 49 87 72 173 143 158 38 69 137 151 42
 86 24 106 5 181 2 92 20 93 147 43 34 14 195 189 0 175 130 30 12 132 167
 187 161 13 112 117 32 192 31 44 100 65 108 48 54 77 179 157 188 141 57 63
 79 36 169 53 199 8 163 28 193 10 50 73 160 129 71 96 116 78 133 162 123
 168 119 164 198 26 102 111 6 150 104 109 97 90 61 51 35 41 4 126 140 184
 154 134 3 152 99 59 18 89 55 122 85 146 124 64 40 76 17 62 67 88 115 125
 15 1 105 171 25 84 91 155 170 16]


In [724]:
#deathvotepandas = pandas.read_csv('deathvotes.csv',encoding="utf-8")
#deathvotes = deathvotepandas.values
##print(deathvotes)
#votearray = np.zeros((len(deadpeople),2),dtype=int)
#print(votearray.shape)
#for i in range(0,len(deathvotes[:,0])):
##    print(i,deathvotes[i,0])
#    foundyet = False
#    j = 0
#    while ((not(foundyet)) and (j < len(deadpeople))):
#        if (deadpeople[j,3] == deathvotes[i,0]):
#            votearray[j,0] = deathvotes[i,1]
#            votearray[j,1] = i
#            foundyet = True
#        else:
#            j = j + 1
##print(votearray[votearray[:,1] > 0])
#deadpeople[:,8] = votearray[:,0]
#deadpeople[:,9] = votearray[:,1]
#print('Done!')
##print(deadpeople)
##print(deadpeople[:,9][deadpeople[:,8] > 0])

Done!


In [16]:
savepath = str(os.getcwd())+'/'
#forvotes = deadpeople[:,(3,7,8,9)]
withvotes = deadpeople[:,(3,7,8,9)][deadpeople[:,8] > 0]
#print(forvotes)
other_rows = deadpeople[:,(3,7,8,9)][deadpeople[:,8] == 0]
other_rows[:,3] = 9999
forvotes = np.vstack((withvotes,other_rows))

w = pandas.DataFrame(forvotes[:,:],columns=['name','description','selected_by','sortorder'])
w.sort_values('sortorder')
w.index.name = 'id'
#w
w.to_csv(savepath+'forsurvey.csv',encoding="utf-8")
#y = pandas.DataFrame(deadpeople,columns=['text','url','death_date','name','age','nationality','cause','description','votes'])
#y.index.name = 'id'
#y.to_csv(savepath+'forvotes.csv',encoding="utf-8")
print('done')

done


In [17]:
savepath = str(os.getcwd())+'/data/'
#print(savepath)
nFiles = int(np.ceil(len(deadpeople)/1000))
for i in range(0,nFiles):
    filename = savepath+'deathlist'+str(i)+'.csv'
    j = int(np.ceil((i+1)*1000))
    priorj = j - 1000
    if (j > len(deadpeople)):
        j = len(deadpeople)
    x = deadpeople[priorj:j]
    y = pandas.DataFrame(x,columns=['text','url','death_date','name','age','nationality','cause','description','selected_by','sortorder'])
    y.index.name = 'id'
    y['cause'] = y['cause'].replace(np.nan,'',regex=True)
    y.to_csv(filename,encoding="utf-8")
    print(filename)
#    print(len(x))
#    print(filename)
#print(nFiles)
#x1 = deadpeople[]

#x = pandas.DataFrame(deadpeople,columns=['text','url','death_date','name','age','nationality','claim','cause'])
#x = pandas.DataFrame(deadpeople,columns=['name','article_length','versions','editors','inbound_links'])#,encoding="utf-8")
#x.index.name = 'id'
#x['cause'] = x['cause'].replace(np.nan,'',regex=True)
#outfile = str(os.getcwd())+'/deathlist.csv'
#x.to_csv(outfile,encoding="utf-8")
print('Done!')

/home/idies/workspace/persistent/fame/data/deathlist0.csv
/home/idies/workspace/persistent/fame/data/deathlist1.csv
/home/idies/workspace/persistent/fame/data/deathlist2.csv
/home/idies/workspace/persistent/fame/data/deathlist3.csv
/home/idies/workspace/persistent/fame/data/deathlist4.csv
/home/idies/workspace/persistent/fame/data/deathlist5.csv
/home/idies/workspace/persistent/fame/data/deathlist6.csv
/home/idies/workspace/persistent/fame/data/deathlist7.csv
/home/idies/workspace/persistent/fame/data/deathlist8.csv
/home/idies/workspace/persistent/fame/data/deathlist9.csv
/home/idies/workspace/persistent/fame/data/deathlist10.csv
/home/idies/workspace/persistent/fame/data/deathlist11.csv
Done!


In [ ]:
# columns in deadpeople:
##  0: raw text
##  1: link
##  2: death date
##  3: name (parsed)
##  4: age (parsed)
##  5: nationality (parsed)
#for i in range(0,len(deadpeople)):

#for i in range(0,len(deadpeople)):
#    claim = ''
#    total_text_length = len(deadpeople[i,0])
#    name_length = len(deadpeople[i,3])
    
#    removename = deadpeople[i,0][name_length+1:total_text_length]

#    if (removename.find(',') <= 6):
#        descriptor = removename[removename.find(',')+1:len(removename)]        
#    else:
#        descriptor = deadpeople[i,0][deadpeople[i,0].find(',')+1:len(deadpeople[i,0])]
#    descriptor = re.sub(trimmer,'',descriptor)

#    nationality_length = len(deadpeople[i,5])
#    descriptor = descriptor[nationality_length:len(descriptor)]

#    if (re.search(entryend,descriptor)):
#        descriptor = descriptor[0:re.search(entryend,descriptor).start()]
        
# parens provide more information on the main claim to fame. So trim off parens.
#    if (descriptor.find('(') != -1):
#        part1 = descriptor[0:descriptor.find('(')-1]
#        part1b = descriptor[len(part1)+2:len(descriptor)]
#        part2 = part1b[part1b.find(')')+1:len(part1b)]
#        descriptor = part1 + part2

# do it again if needed
#    if (descriptor.find('(') != -1):
#        part1 = descriptor[0:descriptor.find('(')-1]
#        part1b = descriptor[len(part1)+2:len(descriptor)]
#        part2 = part1b[part1b.find(')')+1:len(part1b)]
#        descriptor = part1 + part2

# if there is an oxford comma, we need the stuff after the and as well
#    if (oxfordcomma.search(descriptor)):
#        beforeand = descriptor.split('and ')[0]
#        afterand = descriptor.split('and ')[1]
#        if (afterand.find(',') != -1):
#            afterand = afterand[0:afterand.find(',')]
#        claim = beforeand + 'and ' + afterand
    
#    if (descriptor.find(',') != -1):
#        claim = descriptor[0:descriptor.find(',')]
        
#    if (claim == ''):
#        claim = descriptor

#    claim = re.sub(trimmer,'',claim)

#    deadpeople[i,6] = claim

#deadpeople[:,6][deadpeople[:,3] == 'Andrew Chan'] = 'Bali Nine'
#deadpeople[:,6][deadpeople[:,3] == 'Myuran Sukumaran'] = 'Bali Nine'
#deadpeople[:,6][deadpeople[:,3] == 'Robert Abajyan'] = 'Nagorno-Karabakh Defense Army'
#deadpeople[:,6][deadpeople[:,3] == 'Kyaram Sloyan'] = 'Nagorno-Karabakh Defense Army'
#deadpeople[:,6][deadpeople[:,3] == 'Samid Imanov'] = 'Azerbaijani Special Forces'
#deadpeople[:,6][deadpeople[:,3] == 'Murad Mirzayev'] = 'Azerbaijani Special Forces'

#clearthese = [', American food scientist','-American actor','-American theoretical physicist']
#clearthese += ['-Australian nephrologist','-Australian architect','-British history professor']
#clearthese += ['-Israeli rabbi','-Mexican geophysicist','28.','29.','-British television continuity announcer']
#for m in clearthese:
#    deadpeople[:,6][deadpeople[:,6] == m] = ''
    
#deadpeople[:,6][deadpeople[:,6] == 'basketball coachMinnesota Golden Gophers'] = 'basketball coach'
#deadpeople[:,6][deadpeople[:,6] == 'footballerClyde'] = 'footballer'
#deadpeople[:,6][deadpeople[:,6] == 'footballerCrewe Alexandra'] = 'footballer'
#deadpeople[:,6][deadpeople[:,6] == 'football playerNew England Patriots'] = 'football player'
#deadpeople[:,6][deadpeople[:,6] == 'ian Hindi poet'] = 'Hindi poet'
#deadpeople[:,6][deadpeople[:,6] == 'songwriter Another Somebody Done Somebody Wrong Song") and record producer'] = 'songwriter and record producer'

#print('done')
#x = pandas.DataFrame(deadpeople[:,[0,3,5,6]],columns=['text','name','nationality','claim'])
#print(deadpeople)
#x.sort_values('claim')

#x[x['claim'] == '–Papua New Guinean librarian and government advisor']
#y = x.groupby('claim').size()
#y[y > 100].sort_values(ascending=False)
#y[(y >= 8) & (y < 10)].sort_values(ascending=False)
#y[y == 1][2490:2540]